In [1]:
#Загружаем данные и библиотеки для работы с ними:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
df = pd.read_csv('main_task_new.csv')

In [2]:
#Посмотрим на формат типичного значения в каждой колонке:
for column in df.columns:
    print(column, ':', df[column][0])

Restaurant_id : id_5569
City : Paris
Cuisine Style : ['European', 'French', 'International']
Ranking : 5570.0
Rating : 3.5
Price Range : $$ - $$$
Number of Reviews : 194.0
Reviews : [['Good food at your doorstep', 'A good hotel restaurant'], ['12/31/2017', '11/20/2017']]
URL_TA : /Restaurant_Review-g187147-d1912643-Reviews-R_Yves-Paris_Ile_de_France.html
ID_TA : d1912643


In [5]:
#Замечаем, что согласно здравому смыслу, URL и идентификатор в Trip Advisor у каждого ресторана 
#должны быть уникальны, а от колонки, сплошь состоящей из уникальных значений, пользы будет никакой,
#так что эти колонки лучше удалить. Колонку Restaurant_id можно оставить, так как в ней - идентификаторы
#сетей ресторанов, а не каждого отдельного ресторана.
#Принимая это во внимание, отделяем целевую переменную (y) от остальных данных (X).
X = df.drop(['Rating', 'URL_TA', 'ID_TA'], axis = 1)  
y = df['Rating']

In [6]:
#Заполняем пропуски самым популярным значением:
for column in X.columns:
    X[column] = X[column].fillna(X[column].value_counts().index[0])

In [7]:
#Чуть не забыл зафиксировать random seed:
RANDOM_SEED = 42

In [8]:
#Для начала займёмся колонкой с диапазонами цен. Разберёмся, что там вообще за значения:
X['Price Range'].unique()

array(['$$ - $$$', '$$$$', '$'], dtype=object)

In [9]:
#Всего три вида значений, и, очевидно, '$' означает низкие цены,
#'$$ - $$$' - средние, а '$$$$' - высокие. Обычная ординальная переменная, которую можно представить
#в числовом формате, превратив '$' в нули, '$$ - $$$' - в единицы, а '$$$$' - в двойки.
def price(value):
    if value == '$':
        return 0
    elif value == '$$ - $$$':
        return 1
    elif value == '$$$$':
        return 2
X['Price Range'] = X['Price Range'].apply(price)

In [10]:
#Посмотрим, информация о скольки ресторанных сетях у нас есть:
len(X['Restaurant_id'].unique())

11909

In [11]:
#Почти 12 тысяч ресторанных сетей - это, конечно, слишком, слишком много для создания дамми.
#Давайте оставим информацию о 20 самых крупных сетях, остальные объединим под обозначением "Other networks".
all_networks = X['Restaurant_id'].value_counts()
top_networks = all_networks[:20].index

def top_network(network):
    if network in top_networks:
        return network
    else:
        return 'Other networks'
    
X['Restaurant_id'] = X['Restaurant_id'].apply(top_network)

In [12]:
#А теперь можно создать дамми для ресторанных сетей и добавить их к нашей модели:
network_dummies = pd.get_dummies(X['Restaurant_id'])
for column in network_dummies.columns:
    X[column] = network_dummies[column]

In [13]:
#Информация о странах, в которых находятся города(взята из учебника географии :-))

countries = {'France':['Paris', 'Lyon'],'Sweden':['Stockholm'],'United Kingdom':['London', 'Edinburgh'], 
            'Germany':['Berlin', 'Hamburg', 'Munich'], 'Spain':['Madrid', 'Barcelona'], 
            'Italy':['Rome', 'Milan'], 'Slovenia':['Ljubljana'], 'Austria':['Vienna'], 'Portugal':['Lisbon', 'Oporto'], 
            'Ireland':['Dublin'], 'Switzerland':['Zurich', 'Geneva'], 'Belgium':['Brussels'],
            'Poland':['Warsaw', 'Krakow'], 'Hungary':['Budapest'], 'Denmark':['Copenhagen'], 
            'Netherlands':['Amsterdam'], 'Czechia':['Prague'], 'Norway':['Oslo'],
            'Finland':['Helsinki'], 'Slovakia':['Bratislava'], 'Greece':['Athens'], 'Luxembourg':['Luxembourg']}

#Добавим её к нашей модели:

def return_country(city):
    for country in countries:
        if city in countries[country]:
            return country
            break
            
X['Country'] = X['City'].apply(return_country)

In [14]:
#Информация о численности населения городов(взята из Википедии, выражена в тысячах людей):

city_populations = {'Paris':2244, 'Stockholm':960, 'London':8817, 'Berlin':3520, 'Munich':1450, 'Oporto':238,
       'Milan':1389, 'Bratislava':426, 'Vienna':1889, 'Rome':2856, 'Barcelona':1620, 'Madrid':3223,
       'Dublin':1173, 'Brussels':177, 'Zurich':403, 'Warsaw':1794, 'Budapest':1779, 'Copenhagen':613,
       'Amsterdam':873, 'Lyon':484, 'Hamburg':1747, 'Lisbon':506, 'Prague':1300, 'Oslo':693,
       'Helsinki':656, 'Edinburgh':488, 'Geneva':199, 'Ljubljana':280, 'Athens':3100,
       'Luxembourg':115, 'Krakow':1300}

#Добавим её к нашей модели:

def city_population(city):
    return city_populations[city]

X['City population, thousands'] = X['City'].apply(city_population)

In [15]:
#Информация о численности населения интересующих нас стран (в тысячах людей, на 2018 год, с Википедии):

country_populations = {'France':64991, 'United Kingdom':67142, 'Germany':83124,
                       'Luxembourg':604, 'Netherlands':17060, 'Austria':8891,
                       'Belgium':11482, 'Ireland':4819, 'Switzerland':8526,
                       'Greece':10522, 'Italy':60627, 'Portugal':10256, 'Slovenia':2078, 'Spain':46693,
                       'Czechia':10666, 'Hungary':9708, 'Poland':37922, 'Slovakia':5453,
                       'Denmark':5752, 'Finland':5523, 'Norway':5338, 'Sweden':9972}

#Добавим её в датасет:

def country_population(country):
    return country_populations[country]

X['Country population, thousands'] = X['Country'].apply(country_population)

In [16]:
#Информация о средних доходах жителей этих городов(в евро?):

city_incomes = {'Paris':3203.08, 'Stockholm':3331.89, 'London':4180.91, 'Berlin':3423.77, 'Munich':3640.46, 'Oporto':238,
       'Milan':1975.40, 'Bratislava':1333.71, 'Vienna':2470.54, 'Rome':1684.64, 'Barcelona':1928.85, 'Madrid':1933.47,
       'Dublin':3321.19, 'Brussels':2845.81, 'Zurich':7687.71, 'Warsaw':1105.76, 'Budapest':932.90, 'Copenhagen':3739.39,
       'Amsterdam':3711.59, 'Lyon':2748.17, 'Hamburg':3140.50, 'Lisbon':1129.83, 'Prague':1586.31, 'Oslo':3718.89,
       'Helsinki':2822.70, 'Edinburgh':2827.26, 'Geneva':6706.51, 'Ljubljana':1453.95, 'Athens':896.22,
       'Luxembourg':4490.63, 'Krakow':981.58}

#Добавим её в датасет:

def give_city_incomes(city):
    return city_incomes[city]

X['City average incomes'] = X['City'].apply(give_city_incomes)

In [17]:
#А теперь займёмся созданием dummy-переменных для городов и стран:
city_dummies = pd.get_dummies(X['City'])
for column in city_dummies.columns:
    X[column] = city_dummies[column]
    
country_dummies = pd.get_dummies(X['Country'])
for column in country_dummies.columns:
    X[column] = country_dummies[column]

In [19]:
#Каждая строка в колонке Cuisine Style на самом деле представляет собой данные в виде простой текстовой строки.
#Это плохо, нужно превратить эти данные в список кухонь:
X['Cuisine Style'] = X['Cuisine Style'].apply(lambda x:x.split(sep = ', '))

In [20]:
#Избавимся от всех лиших символов - квадратных скобок и кавычек:
def symbol_remover(item):
    new_item = []
    for string in item:
        new_string = string
        for symbol in ['[', ']', "'"]:
            new_string = new_string.replace(symbol, '')
        new_item.append(new_string)
    return new_item

X['Cuisine Style'] = X['Cuisine Style'].apply(symbol_remover)

In [21]:
#Типов кухонь много. Я не хочу загружать свою первую модель таким большим количеством дамми,
#поэтому я решил разделить все эти типы кухонь на категории:
categories = {
'European cuisine' : ['Austrian', 'Belgian','British','Danish','Dutch',
                       'European','French','German','Irish','Italian','Mediterranean','Norwegian','Portuguese',
                       'Scandinavian', 'Scottish','Spanish','Swedish', 'Swiss','Welsh', 'Albanian','Armenian', 'Balti', 
                       'Central European','Czech','Croatian','Eastern European', 'Greek','Hungarian','Latvian',
                       'Mediterranean','Polish','Romanian','Russian','Slovenian','Ukrainian'],
'West&Central Asian Cuisine' :['Afghani','Arabic','Armenian','Azerbaijani','Asian','Central Asian',
                               'Georgian','Egyptian','Israeli','Lebanese','Middle Eastern','Minority Chinese','Mongolian',
                               'Persian', 'Turkish', 'Uzbek', 'Xinjiang'],
'East Asian Cuisine': ['Asian','Bangladeshi','Burmese','Cambodian','Chinese','Filipino','Fujian','Indian', 'Indonesian',
                       'Malaysian','Japanese','Korean','Minority Chinese','Nepali','Pakistani','Sri Lankan',
                       'Singaporean','Taiwanese','Thai','Tibetan','Vietnamese','Yunnan'],
'American Cuisine' : ['American', 'Argentinean','Brazilian','Cajun & Creole', 'Canadian','Caribbean','Central American',
                      'Chilean', 'Colombian','Cuban','Ecuadorean','Jamaican','Latin','Mexican','Native American',
                      'Peruvian','Salvadoran','South American', 'Southwestern', 'Venezuelan'],
'Other Cuisine' : ['African','Ethiopian','Moroccan','Tunisian','Hawaiian','Polynesian','Fusion','International'],
'Drinking Place' : ['Bar','Brew Pub','Delicatessen','Gastropub','Pub','Wine Bar'],
'Meat House' : ['Barbecue','Grill','Steakhouse'],
'Seafood' : ['Seafood','Sushi'],
'Diner' : ['Diner','Soups'],
'Fast food' : ['Cafe','Fast Food','Pizza','Street Food',],
'Vegetarian' : ['Vegan Options','Vegetarian Friendly'],
'Religious' : ['Halal','Kosher']
}

#Упрощаем колонку с типами кухонь, группируя значения в них:
def list_simplifier(cuisines):
    cuisine_list = []
    for cuisine in cuisines:
        for category in categories:
            if cuisine in categories[category] and category not in cuisine_list:
                cuisine_list.append(category)
    return cuisine_list

X['Cuisine Style'] = X['Cuisine Style'].apply(list_simplifier)

In [22]:
#Теперь создадим дамми для типов кухонь:
X['Cuisine Style'] = X['Cuisine Style'].apply(lambda x: pd.Series(x)) #Вначале превратим списку кухонь в объекты Series.

cuisine_dummies = pd.get_dummies(X['Cuisine Style'])
for column in cuisine_dummies.columns:
    X[column] = cuisine_dummies[column]

<ipython-input-22-fd411158e361>:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  X['Cuisine Style'] = X['Cuisine Style'].apply(lambda x: pd.Series(x)) #Вначале превратим списку кухонь в объекты Series.


In [23]:
#Перейдём к колонке с отзывами. Что нас здесь интересует в первую очередь - дата оставления последнего отзыва.
#Превратим её из строк в списки:

X['Reviews'] = X['Reviews'].apply(lambda x:x.split(sep = ', '))

In [24]:
#Избавимся от лишних квадратных скобок и кавычек и здесь:
X['Reviews'] = X['Reviews'].apply(symbol_remover)

In [25]:
#Перведём даты в формат datetime и вынесем их в отдельную колонку:
def date_parser(datecolumn):
    dates_strings = []
    dates = []
    for item in datecolumn:
        for digit in ['0','1','2','3','4','5','6','7','8','9']:
            if digit in item and len(item)>5 and item[2]=='/' and item[5]=='/':
                    if item not in dates:
                        dates_strings.append(item)
                        break
    for date in dates_strings:
        new_date = datetime.strptime(date, '%m/%d/%Y')
        dates.append(new_date)
    return dates

X['Review dates'] = X['Reviews'].apply(date_parser)

In [26]:
#Какую отсюда можно вынести информацию?
#Конечно, можно вынести отсюда дату оставления последнего отзыва.
#Также можно вынести информацию о количестве дней между оставлением последнего и предпоследнего отзывов.
def latest_review(date_pair):
    if len(date_pair)==0:
        return None
    elif len(date_pair)==1:
        return date_pair[0]
    else:
        if date_pair[0]>date_pair[1]:
            return date_pair[0]
        else:
            return date_pair[1]
    
def days_between_reviews(date_pair):
    if len(date_pair) in [0, 1]:
        return None
    else:
        if date_pair[0]>date_pair[1]:
            return (date_pair[0]-date_pair[1]).days
        else:
            return (date_pair[1]-date_pair[0]).days
    
X['Latest review'] = X['Review dates'].apply(latest_review)
X['Days between two latest reviews'] = X['Review dates'].apply(days_between_reviews)

In [27]:
#Представим дату оставления последнего отзыва в виде трёх отдельных признаков - года, месяца и дня:
def lr_year(date):
    return date.year

def lr_month(date):
    return date.month

def lr_day(date):
    return date.day

X['Latest review year'] = X['Latest review'].apply(lr_year)
X['Latest review month'] = X['Latest review'].apply(lr_month)
X['Latest review day'] = X['Latest review'].apply(lr_day)

In [31]:
#В новосозданных признаках должны быть пропуски, так как в колонке Reviews были пустые списки.
#Заполним и их:
for column in ['Latest review year', 'Latest review month', 'Latest review day', 'Days between two latest reviews']:    
    X[column] = X[column].fillna(X[column].value_counts().index[0])

In [28]:
#Модель почти готова. Остаётся лишь удалить лишние теперь категориальные переменные:
X = X.drop(['Reviews', 'Review dates', 'Latest review', 'Restaurant_id', 'Country', 'Cuisine Style', 'City'], axis = 1)

In [33]:
#Модель готова. Можно начинать тестовый запуск:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
regr = RandomForestRegressor(n_estimators=100)
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)

print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.20821149999999997
